### Transformer

In [1]:
from datasets import load_dataset
from transformers import BertModel
from transformers import pipeline
import pandas as pd
import numpy as np

/Users/ling/miniconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The provided code initializes a BERT (Bidirectional Encoder Representations from Transformers) model by loading a pre-trained version known as 'bert-base-uncased' from the Transformers package. BERT is a widely used natural language processing model that can be further explored for different embedding and attention layers, allowing for fine-tuned analysis and manipulation of text data.

In [2]:
model = BertModel.from_pretrained('bert-base-uncased')

In [6]:
len(model.encoder.layer) 

12

In [8]:
# Get all of the model's parameters as a list of tuples.
named_params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(named_params)))

print('==== Embedding Layer ====\n')
for p in named_params[0:5]:
    print("{:<55} {:>32}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Encoder ====\n')
for p in named_params[5:21]:
    print("{:<55} {:>32}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')
for p in named_params[-2:]:
    print("{:<55} {:>32}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 199 different named parameters.

==== Embedding Layer ====

embeddings.word_embeddings.weight                                           (30522, 768)
embeddings.position_embeddings.weight                                         (512, 768)
embeddings.token_type_embeddings.weight                                         (2, 768)
embeddings.LayerNorm.weight                                                       (768,)
embeddings.LayerNorm.bias                                                         (768,)

==== First Encoder ====

encoder.layer.0.attention.self.query.weight                                   (768, 768)
encoder.layer.0.attention.self.query.bias                                         (768,)
encoder.layer.0.attention.self.key.weight                                     (768, 768)
encoder.layer.0.attention.self.key.bias                                           (768,)
encoder.layer.0.attention.self.value.weight                                   (768, 768)
encod

### Search semantics

The Hugging Face summarization task page lists models that support summarization. In this section, we will the following resources:

- data: We will work with the 'xsum dataset,' containing a collection of BBC articles and their corresponding summaries. This dataset serves as the foundation for our tasks.

- model: Our chosen model is the 't5-small model,' with 60 million parameters (equivalent to 242MB for PyTorch). T5, an encoder-decoder model developed by Google, boasts versatility, supporting various tasks including summarization, translation, question-answering, and text classification.

In [3]:
xsum_dataset = load_dataset(
    "xsum", version="1.2.0"
)  

Found cached dataset xsum (/Users/ling/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 143.52it/s]


This dataset provides 3 columns:

- document: the BBC article text
- summary: a "ground-truth" summary --> Note how subjective this "ground-truth" is. Is this the same summary you would write? This a great example of how many LLM applications do not have obvious "right" answers.
- id: article ID

In [4]:
# Taking a sample of 100 rows
xsum_sample = xsum_dataset["train"].select(range(100)).to_pandas()

# Combining 'document' and 'summary' columns
xsum_sample["combined"] = (
    "Document: " + xsum_sample.document.str.strip() + "; Summary: " + xsum_sample.summary.str.strip()
)

In [5]:
from sentence_transformers import SentenceTransformer

#encoding the data
encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
encoded_data = encoder.encode(xsum_sample["combined"])

In [6]:
encoded_data

array([[-0.12973621, -0.07995631, -0.02103529, ...,  0.01458149,
        -0.04181127,  0.05969846],
       [-0.10183057, -0.00813391,  0.01535374, ...,  0.0399589 ,
        -0.10245837,  0.08624614],
       [-0.06544376, -0.22466165,  0.01042661, ...,  0.06865784,
         0.07314408,  0.01244242],
       ...,
       [-0.02983218,  0.0890516 , -0.07677924, ...,  0.01902936,
        -0.02879047,  0.09262642],
       [ 0.00134849, -0.07791834,  0.02811532, ...,  0.18549399,
         0.0806486 , -0.11673684],
       [-0.17274612,  0.04478129,  0.00123185, ...,  0.10003569,
         0.03951619,  0.03509405]], dtype=float32)

In [9]:
import faiss

index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
faiss.normalize_L2(encoded_data)
index.add_with_ids(encoded_data, np.arange(len(encoded_data)))

In [10]:
search_text = xsum_sample["combined"][0]
search_text

'Document: The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate 

In [11]:
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

k = 2
distances, ann = index.search(_vector, k=k)
results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})

In [12]:
distances

array([[1.       , 0.6326008]], dtype=float32)

In [13]:
xsum_sample["summary"][results['ann'][0]]

'Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.'

In [14]:
xsum_sample["summary"][results['ann'][1]]

'A 13ft-deep hole causing widespread disruption to trains in south London has been found to have opened above an active sewer.'

### Summarization

In [19]:
summarizer = pipeline(
    task="summarization",
    model="t5-small",
    min_length=20,
    max_length=60,
    truncation=True,
) 

In [20]:
xsum_sample["summary"][0]

'Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.'

In [21]:
summarizer(xsum_sample["document"][0])

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining wall, flooding many commercial properties .'}]

In [21]:
# Apply to a batch of articles
results = summarizer(xsum_sample["document"])

In [22]:
display(
    pd.DataFrame.from_dict(results)
    .rename({"summary_text": "generated_summary"}, axis=1)
    .join(pd.DataFrame.from_dict(xsum_sample))[
        ["generated_summary", "summary", "document"]
    ]
)

generated_summary  \
0  the full cost of damage in Newton Stewart is s...   
1  a fire alarm went off at the Holiday Inn in Ho...   
2  Sebastian Vettel will start third ahead of tea...   
3  the 67-year-old is accused of committing the o...   
4  a man receiving psychiatric treatment at the c...   
5  Gregor Townsend gave a debut to powerhouse win...   
6  Veronica Vanessa Chango-Alverez, 31, was kille...   
7  the 25-year-old was hit by a motorbike during ...   
8  gundogan will not be fit for the start of the ...   
9  the crash happened about 07:20 GMT at the junc...   

                                             summary  \
0  Clean-up operations are continuing across the ...   
1  Two tourist buses have been destroyed by fire ...   
2  Lewis Hamilton stormed to pole position at the...   
3  A former Lincolnshire Police officer carried o...   
4  An armed man who locked himself into a room at...   
5  Defending Pro12 champions Glasgow Warriors bag...   
6  A man with links to a car that was involved in...   
7  Welsh cyclist Luke Rowe says changes to the sp...   
8  Manchester City midfielder Ilkay Gundogan says...   
9  A jogger has been hit by an unmarked police ca...   

                                            document  
0  The full cost of damage in Newton Stewart, one...  
1  A fire alarm went off at the Holiday Inn in Ho...  
2  Ferrari appeared in a position to challenge un...  
3  John Edward Bates, formerly of Spalding, Linco...  
4  Patients and staff were evacuated from Cerahpa...  
5  Simone Favaro got the crucial try with the las...  
6  Veronica Vanessa Chango-Alverez, 31, was kille...  
7  Belgian cyclist Demoitie died after a collisio...  
8  Gundogan, 26, told BBC Sport he "can see the f...  
9  The crash happened about 07:20 GMT at the junc...

### Search and sampling in inference

You may see parameters like `num_beams`, `do_sample`, etc. specified in Hugging Face pipelines.  These are inference configurations.

LLMs work by predicting (generating) the next token, then the next, and so on.  The goal is to generate a high probability sequence of tokens, which is essentially a search through the (enormous) space of potential sequences.

To do this search, LLMs use one of two main methods:
* **Search**: Given the tokens generated so far, pick the next most likely token in a "search."
   * **Greedy search** (default): Pick the single next most likely token in a greedy search.
   * **Beam search**: Greedy search can be extended via beam search, which searches down several sequence paths, via the parameter `num_beams`.
* **Sampling**: Given the tokens generated so far, pick the next token by sampling from the predicted distribution of tokens.
   * **Top-K sampling**: The parameter `top_k` modifies sampling by limiting it to the `k` most likely tokens.
   * **Top-p sampling**: The parameter `top_p` modifies sampling by limiting it to the most likely tokens up to probability mass `p`.

You can toggle between search and sampling via parameter `do_sample`.

For more background on search and sampling, see [this Hugging Face blog post](https://huggingface.co/blog/how-to-generate).

We will illustrate these various options below using our summarization pipeline.

In [23]:
# We can instead do a beam search by specifying num_beams.
summarizer(xsum_sample["document"][0], num_beams=10)

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining wall, flooding many commercial properties .'}]

In [25]:
# Alternatively, we could use sampling.
summarizer(xsum_sample["document"][0], do_sample=True)

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the waters breached a retaining wall, flooding many commercial properties .'}]

In [26]:
# We can modify sampling to be more greedy by limiting sampling to the top_k or top_p most likely next tokens.
summarizer(xsum_sample["document"][0], do_sample=True, top_k=10, top_p=0.8)›

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining wall, flooding many commercial properties .'}]

In [27]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from rich import print

In [28]:
# Load the pre-trained tokenizer and model.
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")


# For summarization, T5-small expects a prefix "summarize: "
# so we prepend that to each article as a prompt.

articles = list(map(lambda article: "summarize: " + article, xsum_sample["document"]))

# Tokenize the input
inputs = tokenizer(
    articles, max_length=1024,
    return_tensors="pt", padding=True, truncation=True
)
# Generate summaries
summary_ids = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    num_beams=2,
    min_length=0,
    max_length=60,
)
# Decode the generated summaries
decoded_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

print(decoded_summaries[0])

the full cost of damage in Newton Stewart is still being assessed. many roads in peeblesshire remain badly affected
by standing water. the water breached a retaining wall, flooding many commercial properties.

### Zero-shot classification

Zero-shot classification (or zero-shot learning) is the task of classifying a piece of text into one of a few given categories or labels, without having explicitly trained the model to predict those categories beforehand. The idea appeared in literature before modern LLMs, but recent advances in LLMs have made zero-shot learning much more flexible and powerful.

Background reading: See the Hugging Face task page on zero-shot classification or Wikipedia on zero-shot learning.

In this section, we will use:

Data: a few example articles from the xsum dataset used in the Summarization section above. Our goal is to label news articles under a few categories.
Model: nli-deberta-v3-small, a fine-tuned version of the DeBERTa model. The DeBERTa base model was developed by Microsoft and is one of several models derived from BERT; for more details on DeBERTa, see the Hugging Face doc page, the code on GitHub, or the research paper.

In [11]:
zero_shot_pipeline = pipeline(
    task="zero-shot-classification",
    model="cross-encoder/nli-deberta-v3-small",
)


def categorize_article(article: str) -> None:
    """
    This helper function defines the categories (labels) which the model must use to label articles.
    Note that our model was NOT fine-tuned to use these specific labels,
    but it "knows" what the labels mean from its more general training.

    This function then prints out the predicted labels alongside their confidence scores.
    """
    results = zero_shot_pipeline(
        article,
        candidate_labels=[
            "politics",
            "finance",
            "sports",
            "science and technology",
            "pop culture",
            "breaking news",
        ],
    )
    # Print the results nicely
    del results["sequence"]
    display(pd.DataFrame(results))

/Users/linghuang/miniconda3/envs/langchain/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [12]:
categorize_article(
    """
A large language model (LLM) is a language model notable for its ability to achieve general-purpose language understanding and generation. LLMs acquire these abilities by learning statistical relationships from text documents during a computationally intensive self-supervised and semi-supervised training process.[1] LLMs are artificial neural networks following a transformer architecture.[2]

They can be used for text generation by taking an input text and repeatedly predicting the next token or word.[3] Up to 2020, fine tuning was the only way a model could be adapted to be able to accomplish specific tasks. Larger sized models, such as GPT-3, however, can be prompt-engineered to achieve similar results.[4] They are thought to acquire knowledge about syntax, semantics and "ontology" inherent in human language corpora, but also inaccuracies and biases present in the corpora.[5]

Notable examples include OpenAI's GPT models (e.g., GPT-3.5 and GPT-4, used in ChatGPT), Google's PaLM (used in Bard), and Meta's LLaMA, as well as BLOOM, Ernie 3.0 Titan, and Anthropic's Claude 2.
"""
)

labels    scores
0  science and technology  0.762033
1           breaking news  0.089980
2             pop culture  0.059526
3                 finance  0.036244
4                  sports  0.027556
5                politics  0.024660

### Few-shot learning

In few-shot learning tasks, you give the model an instruction, a few query-response examples of how to follow that instruction, and then a new query. The model must generate the response for that new query. This technique has pros and cons: it is very powerful and allows models to be reused for many more applications, but it can be finicky and require significant prompt engineering to get good and reliable results.

Background reading: See the Wikipedia page on few-shot learning or this Hugging Face blog about few-shot learning.

In this section, we will use:

- Task: Few-shot learning can be applied to many tasks. Here, we will do sentiment analysis, which was covered earlier. However, you will see how few-shot learning allows us to specify custom labels, whereas the previous model was tuned for a specific set of labels. We will also show other (toy) tasks at the end. In terms of the Hugging Face task specified in the pipeline constructor, few-shot learning is handled as a text-generation task.
- Data: We use a few examples, including a tweet example from the blog post linked above.
- Model: gpt-neo-1.3B, a version of the GPT-Neo model discussed in the blog linked above. It is a transformer model with 1.3 billion parameters developed by Eleuther AI. For more details, see the code on GitHub or the research paper.

In [21]:
# We will limit the response length for our few-shot learning tasks.
few_shot_pipeline = pipeline(
    task="text-generation",
    model="EleutherAI/gpt-neo-1.3B",
    max_new_tokens=10,
    # model_kwargs={"cache_dir": DA.paths.datasets},
)

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [22]:
# Get the token ID for "###", which we will use as the EOS token below.
eos_token_id = few_shot_pipeline.tokenizer.encode("###")[0]

In [23]:
# Without any examples, the model output is inconsistent and usually incorrect.
results = few_shot_pipeline(
    """For each tweet, describe its sentiment:

[Tweet]: "This new music video was incredible"
[Sentiment]:""",
    eos_token_id=eos_token_id,
)

print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


For each tweet, describe its sentiment:

[Tweet]: "This new music video was incredible"
[Sentiment]: "Very positive"

And the sentiment:


In [24]:
# With only 1 example, the model may or may not get the answer right.
results = few_shot_pipeline(
    """For each tweet, describe its sentiment:

[Tweet]: "This is the link to the article"
[Sentiment]: Neutral
###
[Tweet]: "This new music video was incredible"
[Sentiment]:""",
    eos_token_id=eos_token_id,
)

print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


For each tweet, describe its sentiment:

[Tweet]: "This is the link to the article"
[Sentiment]: Neutral
###
[Tweet]: "This new music video was incredible"
[Sentiment]: Strong
###


In [25]:
# With 1 example for each sentiment, the model is more likely to understand!
results = few_shot_pipeline(
    """For each tweet, describe its sentiment:

[Tweet]: "I hate it when my phone battery dies."
[Sentiment]: Negative
###
[Tweet]: "My day has been 👍"
[Sentiment]: Positive
###
[Tweet]: "This is the link to the article"
[Sentiment]: Neutral
###
[Tweet]: "This new music video was incredible"
[Sentiment]:""",
    eos_token_id=eos_token_id,
)

print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


For each tweet, describe its sentiment:

[Tweet]: "I hate it when my phone battery dies."
[Sentiment]: Negative
###
[Tweet]: "My day has been 👍"
[Sentiment]: Positive
###
[Tweet]: "This is the link to the article"
[Sentiment]: Neutral
###
[Tweet]: "This new music video was incredible"
[Sentiment]: Positive
###


In [26]:
# The model isn't ready to serve drinks!
results = few_shot_pipeline(
    """For each food, suggest a good drink pairing:

[food]: tapas
[drink]: wine
###
[food]: pizza
[drink]: soda
###
[food]: jalapenos poppers
[drink]: beer
###
[food]: scone
[drink]:""",
    eos_token_id=eos_token_id,
)

print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


For each food, suggest a good drink pairing:

[food]: tapas
[drink]: wine
###
[food]: pizza
[drink]: soda
###
[food]: jalapenos poppers
[drink]: beer
###
[food]: scone
[drink]: soda
[food]: poppers
###


In [28]:
results = few_shot_pipeline(
    """Given a word describing how someone is feeling, suggest a description of that person.  The description should not include the original word.

[word]: happy
[description]: smiling, laughing, clapping
###
[word]: nervous
[description]: glancing around quickly, sweating, fidgeting
###
[word]: sleepy
[description]: heavy-lidded, slumping, rubbing eyes
###
[word]: confused
[description]:""",
    eos_token_id=eos_token_id,
)
print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


Given a word describing how someone is feeling, suggest a description of that person.  The description should not include the original word.

[word]: happy
[description]: smiling, laughing, clapping
###
[word]: nervous
[description]: glancing around quickly, sweating, fidgeting
###
[word]: sleepy
[description]: heavy-lidded, slumping, rubbing eyes
###
[word]: confused
[description]: wandering around, turning in circles, confused
###


In [29]:
# We override max_new_tokens to generate longer answers.
# These book descriptions were taken from their corresponding Wikipedia pages.
results = few_shot_pipeline(
    """Generate a book summary from the title:

[book title]: "Stranger in a Strange Land"
[book description]: "This novel tells the story of Valentine Michael Smith, a human who comes to Earth in early adulthood after being born on the planet Mars and raised by Martians, and explores his interaction with and eventual transformation of Terran culture."
###
[book title]: "The Adventures of Tom Sawyer"
[book description]: "This novel is about a boy growing up along the Mississippi River. It is set in the 1840s in the town of St. Petersburg, which is based on Hannibal, Missouri, where Twain lived as a boy. In the novel, Tom Sawyer has several adventures, often with his friend Huckleberry Finn."
###
[book title]: "Dune"
[book description]: "This novel is set in the distant future amidst a feudal interstellar society in which various noble houses control planetary fiefs. It tells the story of young Paul Atreides, whose family accepts the stewardship of the planet Arrakis. While the planet is an inhospitable and sparsely populated desert wasteland, it is the only source of melange, or spice, a drug that extends life and enhances mental abilities.  The story explores the multilayered interactions of politics, religion, ecology, technology, and human emotion, as the factions of the empire confront each other in a struggle for the control of Arrakis and its spice."
###
[book title]: "Blue Mars"
[book description]:""",
    eos_token_id=eos_token_id,
    max_new_tokens=50,
)

print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


Generate a book summary from the title:

[book title]: "Stranger in a Strange Land"
[book description]: "This novel tells the story of Valentine Michael Smith, a human who comes to Earth in early adulthood after being born on the planet Mars and raised by Martians, and explores his interaction with and eventual transformation of Terran culture."
###
[book title]: "The Adventures of Tom Sawyer"
[book description]: "This novel is about a boy growing up along the Mississippi River. It is set in the 1840s in the town of St. Petersburg, which is based on Hannibal, Missouri, where Twain lived as a boy. In the novel, Tom Sawyer has several adventures, often with his friend Huckleberry Finn."
###
[book title]: "Dune"
[book description]: "This novel is set in the distant future amidst a feudal interstellar society in which various noble houses control planetary fiefs. It tells the story of young Paul Atreides, whose family accepts the stewardship of the planet Arrakis. While the planet is an in

the full cost of damage in Newton Stewart is still being assessed. many roads in peeblesshire remain badly affected
by standing water. the water breached a retaining wall, flooding many commercial properties.

### Query for similar documents using FAISS

document  \
0  The full cost of damage in Newton Stewart, one...   
1  A fire alarm went off at the Holiday Inn in Ho...   
2  Ferrari appeared in a position to challenge un...   
3  John Edward Bates, formerly of Spalding, Linco...   
4  Patients and staff were evacuated from Cerahpa...   
5  Simone Favaro got the crucial try with the las...   
6  Veronica Vanessa Chango-Alverez, 31, was kille...   
7  Belgian cyclist Demoitie died after a collisio...   
8  Gundogan, 26, told BBC Sport he "can see the f...   
9  The crash happened about 07:20 GMT at the junc...   

                                             summary        id  
0  Clean-up operations are continuing across the ...  35232142  
1  Two tourist buses have been destroyed by fire ...  40143035  
2  Lewis Hamilton stormed to pole position at the...  35951548  
3  A former Lincolnshire Police officer carried o...  36266422  
4  An armed man who locked himself into a room at...  38826984  
5  Defending Pro12 champions Glasgow Warriors bag...  34540833  
6  A man with links to a car that was involved in...  20836172  
7  Welsh cyclist Luke Rowe says changes to the sp...  35932467  
8  Manchester City midfielder Ilkay Gundogan says...  40758845  
9  A jogger has been hit by an unmarked police ca...  30358490